In [28]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters=1000
learning_rate = 3e-4
eval_iters = 250

cuda


In [29]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [30]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [31]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[ 1, 72, 68, 66, 58, 73, 61, 62],
        [ 0, 28, 68, 71, 68, 73, 61, 78],
        [ 1, 59, 58, 54, 71,  1, 73, 61],
        [65, 68, 76,  1, 61, 58, 71, 11]], device='cuda:0')
targets:
tensor([[72, 68, 66, 58, 73, 61, 62, 67],
        [28, 68, 71, 68, 73, 61, 78, 10],
        [59, 58, 54, 71,  1, 73, 61, 54],
        [68, 76,  1, 61, 58, 71, 11,  1]], device='cuda:0')


In [32]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [33]:
class bigramlanguagemodel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
        
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = bigramlanguagemodel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


IUJK﻿YCS I3&UU]QV]CBiFPqhonyW-M'SVk9ftt]s[v80.OWP
(Jk4:xF*D"'[C:EUj8M7O pS&B?yfBto1Zn eeb.0laNB?aLlo"EXjWtEUU﻿;HNzR:!JBa"70lf]UA;eTc-WfnRIt,2d,.o"Q20:NPMAJld:*KBwkNZD* (;-2gmC-)F9RnmK﻿Vg﻿9SU IN﻿
v?iB?
x5r9j'-D9p
r;]LX2HglVDs4:-'*32zrc
:TKMI1;,TZr﻿l.Y)x5wl0X"J7rd]ld
j2A7gC
IA)6j2hmJ?DBlF.CD-7s5F[tCu6q* (JU6Az[JvVAofIos*1frCVg4)Y8QQWtwwO*aUjsQW.3iy0k9OO'.0.e (xzdPYD6-c
P3u:;]﻿Srqq*jeDS*1]?*]nmgL e;]t4z5PsChPpVU 9,uGU B'iuRv[vuRHfu7qyNG1YyI-vC UJRP.tPn[2zvsij(Mio8QmR"pS,_4)(-oB??9vXXZZR?a&hx?A!b9ha


In [34]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.907, val loss: 4.934
step: 250, train loss: 4.872, val loss: 4.867
step: 500, train loss: 4.786, val loss: 4.805
step: 750, train loss: 4.734, val loss: 4.743
5.101407527923584


In [35]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


 gLI32kN1_buu pj5q*o*j1]RA2R"8&Uav_zdDs11!1sh63[u],63.3&LQ(RNWO 5yX2n[j:A;WZbg1fdMSD_KA2cR?zG7v_uEjdjmA9ZaD.A:hFwsA?g1Z 37 wUj*!JvE5kT1V32_,OWZa vO I
ilZ:5UP8oEa03[s5PwkVQmd7rC﻿8Ydi[nwrX5eewwsFU﻿1agrtn0"k1K7Binl)E4PAb]C*.oE[T!v_*4'Jwx7fsMZaELl!BRw6e;AJ".t-pBG[!﻿X(z
peQ5
uHFzwGzKpM&UPYl(DNn(z_c! D&M3IgD"3u*]C
j*rlp71wjKR6s- (ChSq*]gDN29IlmiF9_i(jt8'[3QZ.CijKHo9m(zdDD2JPIx?zxPHoB1cy&]QW*vluHt8seZ ;2tnGtVg8 egrx?J1Z3V(:TLIaBF:6Q04LNOx?]X,Gf_AQs5!FU*gdKb.qzF8YlJwb;ER6﻿,B(R?x﻿AGP&mmvpr1_suiA;JFZbVU9(
